# Vector Stores (aka. Vector Databases)
* Store embeddings in a very fast searchable database.

## Setup

#### After you download the code from the github repository in your computer
In terminal:
* cd project_name
* pyenv local 3.11.4
* poetry install
* poetry shell

#### To open the notebook with Jupyter Notebooks
In terminal:
* jupyter lab

Go to the folder of notebooks and open the right notebook.

#### To see the code in Virtual Studio Code or your editor of choice.
* open Virtual Studio Code or your editor of choice.
* open the project-folder
* open the 004-vector-stores.py file

## Create your .env file
* In the github repo we have included a file named .env.example
* Rename that file to .env file and here is where you will add your confidential api keys. Remember to include:
* OPENAI_API_KEY=your_openai_api_key
* LANGCHAIN_TRACING_V2=true
* LANGCHAIN_ENDPOINT=https://api.smith.langchain.com
* LANGCHAIN_API_KEY=your_langchain_api_key
* LANGCHAIN_PROJECT=your_project_name

We will call our LangSmith project **004-vector-stores**.

## Track operations
From now on, we can track the operations **and the cost** of this project from LangSmith:
* [smith.langchain.com](https://smith.langchain.com)

## Connect with the .env file located in the same directory of this notebook

If you are using the pre-loaded poetry shell, you do not need to install the following package because it is already pre-loaded for you:

In [1]:
#pip install python-dotenv

In [2]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai_api_key = os.environ["OPENAI_API_KEY"]

#### Install LangChain

If you are using the pre-loaded poetry shell, you do not need to install the following package because it is already pre-loaded for you:

In [3]:
#!pip install langchain

## Connect with an LLM

If you are using the pre-loaded poetry shell, you do not need to install the following package because it is already pre-loaded for you:

In [4]:
#!pip install langchain-openai

* NOTE: Since right now is the best LLM in the market, we will use OpenAI by default. You will see how to connect with other Open Source LLMs like Llama3 or Mistral in a next lesson.

In [5]:
from langchain_openai import ChatOpenAI

chatModel = ChatOpenAI(model="gpt-3.5-turbo-0125")

## Reminder: Steps of the RAG process.
* When you load a document, you end up with strings. Sometimes the strings will be too large to fit into the context window. In those occassions we will use the RAG technique:
    * Split document in small chunks.
    * Transform text chunks in numeric chunks (embeddings).
    * **Load embeddings to a vector database (aka vector store)**.
    * Load question and retrieve the most relevant embeddings to respond it.
    * Sent the embeddings to the LLM to format the response properly.

## Vector databases (aka vector stores): store and search embeddings
* See the documentation page [here](https://python.langchain.com/v0.1/docs/modules/data_connection/vectorstores/).
* See the list of vector stores [here](https://python.langchain.com/v0.1/docs/integrations/vectorstores/).

If you are using the pre-loaded poetry shell, you do not need to install the following package because it is already pre-loaded for you:

In [2]:
#!pip install langchain-chroma

     ---------------------------------------- 0.0/67.3 kB ? eta -:--:--
     ---------------------------------------- 67.3/67.3 kB 1.8 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
   ---------------------------------------- 0.0/19.3 MB ? eta -:--:--
   ---------------------------------------- 0.2/19.3 MB 5.6 MB/s eta 0:00:04
    --------------------------------------- 0.5/19.3 MB 5.8 MB/s eta 0:00:04
   - -------------------------------------- 0.7/19.3 MB 5.8 MB/s eta 0:00:04
   -- ------------------------------------- 1.1/19.3 MB 5.7 MB/s eta 0:00:04
   -- ------------------------------------- 1.4/19.3 MB 6.3 MB/s eta 0:00:03
   --- ------------------------------------ 1.8/19.3 MB 6.7 MB/


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: C:\Users\Juan\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [6]:
from langchain_community.document_loaders import TextLoader
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter
from langchain_chroma import Chroma

# Load the document, split it into chunks, embed each chunk and load it into the vector store.
loaded_document = TextLoader('./data/state_of_the_union.txt',encoding='utf-8').load()

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)

chunks_of_text = text_splitter.split_documents(loaded_document)

vector_db = Chroma.from_documents(chunks_of_text, OpenAIEmbeddings())

In [7]:
question = "What did the president say about the John Lewis Voting Rights Act?"

response = vector_db.similarity_search(question)

print(response[0].page_content)

Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. 

Tonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. 

One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. 

And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.


The `.similarity_search` method in the previous code is used to find the most relevant text chunks from a pre-processed document that are similar to a given query. Here's how it works step-by-step

1. **Document Processing and Embedding:**
   - The document `state_of_the_union.txt` is loaded using the `TextLoader` from the LangChain Community package.
   - The document is then split into smaller chunks of text using the `CharacterTextSplitter`, where each chunk is 1000 characters long without any overlap between the chunks.
   - Each chunk of text is then transformed into an embedding using `OpenAIEmbeddings`. Embeddings are high-dimensional vectors that represent the semantic content of the text.
   - These embeddings are stored in an instance of `Chroma`, which serves as a vector database optimized for efficient similarity searches.

2. **Using `.similarity_search`:**
   - When you invoke `vector_db.similarity_search(question)`, the method converts the query (`"What did the president say about the John Lewis Voting Rights Act?"`) into an embedding using the same method that was used for the chunks.
   - It then searches the vector database (`vector_db`) to find the chunks whose embeddings are most similar to the embedding of the query. This similarity is typically measured using metrics such as cosine similarity.
   - The search results are sorted by relevance, with the most relevant chunks (those that are semantically closest to the query) returned first.

3. **Output:**
   - The result of the `.similarity_search` is stored in `response`, which contains the relevant chunks and their similarity scores.
   - The script prints the content of the most relevant chunk (the first result), which should ideally contain information about what the president said regarding the John Lewis Voting Rights Act.

This method is particularly useful in applications like question answering or document retrieval where you need to quickly find the most relevant parts of a large text based on a query.

## How to execute the code from Visual Studio Code
* In Visual Studio Code, see the file 004-vector-stores.py
* In terminal, make sure you are in the directory of the file and run:
    * python 004-vector-stores.py